<a href="https://colab.research.google.com/github/tegveersiyan/NFL-Sports-Betting-Predictor/blob/main/NFL_Sports_Betting_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup


In [ ]:
#Install Packages
!pip install nfl_data_py
!pip install pandas
!pip install os
!pip install matplotlib

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [ ]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

In [ ]:
pbp_data = nfl.import_pbp_data([2021, 2022, 2023, 2024], downcast=True)
seasonal_data = nfl.import_seasonal_data([2021, 2022, 2023, 2024])
weekly_data = nfl.import_weekly_data([2021, 2022, 2023, 2024], downcast=True)

2021 done.
2022 done.
2023 done.
2024 done.
Downcasting floats.
Downcasting floats.


In [ ]:
nfl.see_weekly_cols()
weekly_data['opponent_team']

,opponent_team
0,DAL
1,ATL
2,LA
3,NE
4,MIA
...,...
4696,NYJ
4697,SEA
4698,MIN
4699,SEA


#Handler


In [ ]:
# Converts player name to match abbreviation name in dataset
def parse_name(player_name):
  parts = player_name.split()
  if len(parts) < 2:
    raise ValueError("No valid first or last name")
  first_init = parts[0][0]
  last_name = ' '.join(parts[1:])
  abbr_name = first_init + '.' + last_name
  return abbr_name

def player_stats(player, opp_team, prop_type):
  player_data = weekly_data[weekly_data['player_display_name'] == player]
  p_opp_data = player_data[player_data['opponent_team'] == opp_team]
  prop_data = p_opp_data[prop_type]
  return prop_data

#K Nearest Neighbors
def knn(prop_data, prop_num, k=3):

  if len(prop_data) < 1:
        return "Not enough historical data"

  if len(prop_data) < 3:
        k = 1

  x = prop_data.values.reshape(-1, 1)
  y = (prop_data > prop_num).astype(int)

  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(x, y)

  predictions = knn.predict(x)
  probabilities = knn.predict_proba(x)

  final_prediction = int(np.mean(predictions) > 0.5)

  if probabilities.shape[1] == 1:
      avg_probability = 1.0
  else:
      avg_probability = np.mean([prob[1] if final_prediction == 1 else prob[0] for prob in probabilities])

  historical_accuracy = np.mean(predictions == y)

  avg_value = np.mean(prop_data)
  last_5_avg = np.mean(prop_data[-5:])
  times_over = np.sum(y)

  return {
        'prediction': 'OVER' if final_prediction == 1 else 'UNDER',
        'confidence': f"{avg_probability * 100:.1f}%",
        'historical_accuracy': f"{historical_accuracy * 100:.1f}%",
        'average_yards': f"{avg_value:.1f}",
        'last_5_avg': f"{last_5_avg:.1f}",
        'times_over': f"{times_over} out of {len(prop_data)} games",
        'number_of_games': len(prop_data),
        'k_used': k
    }

#Linear Regression
def linear_regression(prop_data, prop_num):
  if len(prop_data) < 1:
    return "Not enough historical data"

  x = prop_data.values.reshape(-1, 1)
  y = prop_data.values

  model = LinearRegression()
  model.fit(x, y)

  prediction = model.predict(x[-1].reshape(1, -1))[0]
  confidence = abs(0.5 - (prediction - prop_num) / prop_num) * 100

  return {
    'prediction': 'OVER' if prediction > prop_num else 'UNDER',
    'confidence': f"{min(confidence, 100):.1f}%",
    'predicted_yards': f"{prediction:.1f}",
    'average_yards': f"{np.mean(prop_data):.1f}",
    'last_5_avg': f"{np.mean(prop_data[-5:]):.1f}",
    'number_of_games': len(prop_data)
  }

#Random Forest
def random_forest(prop_data, prop_num):
  if len(prop_data) < 1:
    return "Not enough historical data"

  x = prop_data.values.reshape(-1, 1)
  y = prop_data.values

  rf = RandomForestRegressor(n_estimators=100)
  rf.fit(x, y)

  prediction = rf.predict(x[-1].reshape(1, -1))[0]
  confidence = abs(0.5 - (prediction - prop_num) / prop_num) * 100

  return {
    'prediction': 'OVER' if prediction > prop_num else 'UNDER',
    'confidence': f"{min(confidence, 100):.1f}%",
    'predicted_yards': f"{prediction:.1f}",
    'average_yards': f"{np.mean(prop_data):.1f}",
    'last_5_avg': f"{np.mean(prop_data[-5:]):.1f}",
    'number_of_games': len(prop_data)
  }

#Support Vector Machines
def svm(prop_data, prop_num):
  if len(prop_data) < 1:
    return "Not enough historical data"

  x = prop_data.values.reshape(-1, 1)
  y = prop_data.values

  model = SVR(kernel='rbf')
  model.fit(x, y)

  prediction = model.predict(x[-1].reshape(1, -1))[0]
  confidence = abs(0.5 - (prediction - prop_num) / prop_num) * 100

  return {
    'prediction': 'OVER' if prediction > prop_num else 'UNDER',
    'confidence': f"{min(confidence, 100):.1f}%",
    'predicted_yards': f"{prediction:.1f}",
    'average_yards': f"{np.mean(prop_data):.1f}",
    'last_5_avg': f"{np.mean(prop_data[-5:]):.1f}",
    'number_of_games': len(prop_data)
  }

#Neural Network
def neural_network(prop_data, prop_num):
  if len(prop_data) < 1:
    return "Not enough historical data"

  x = prop_data.values.reshape(-1, 1)
  y = prop_data.values

  model = Sequential([
    Dense(16, activation='relu', input_shape=(1,)),
    Dense(8, activation='relu'),
    Dense(1)
  ])

  model.compile(optimizer='adam', loss='mse')
  model.fit(x, y, epochs=100, verbose=0)

  prediction = model.predict(x[-1].reshape(1, -1))[0][0]
  confidence = abs(0.5 - (prediction - prop_num) / prop_num) * 100

  return {
    'prediction': 'OVER' if prediction > prop_num else 'UNDER',
    'confidence': f"{min(confidence, 100):.1f}%",
    'predicted_yards': f"{prediction:.1f}",
    'average_yards': f"{np.mean(prop_data):.1f}",
    'last_5_avg': f"{np.mean(prop_data[-5:]):.1f}",
    'number_of_games': len(prop_data)
  }

def compare_models_test(prop_data, prop_num):
  models = {
    'KNN': knn,
    'Linear Regression': linear_regression,
    'Random Forest': random_forest,
    'SVM': svm,
    'Neural Network': neural_network
  }

  print("\nPredictions from different models:")
  for name, model_func in models.items():
    result = model_func(prop_data, prop_num)
    print(f"\n{name}:")
    for key, value in result.items():
      print(f"{key}: {value}")

def compare_models():
  print("Example: Brock Purdy, SEA, passing_yards, 262.5")

  player = input('Player Name: ')
  opp_team = input('Opponent Team: ')
  prop_type = input('Prop Type: ')
  prop_num = float(input('Prop Number: '))

  prop_data = player_stats(player, opp_team, prop_type)

  models = {
      'KNN': knn,
      'Linear Regression': linear_regression,
      'Random Forest': random_forest,
      'SVM': svm,
      'Neural Network': neural_network
  }

  print("\nPredictions from different models:")
  for name, model_func in models.items():
    result = model_func(prop_data, prop_num)
    print(f"\n{name}:")
    for key, value in result.items():
      print(f"{key}: {value}")

# Handler - Advanced


In [ ]:
def get_features_for_prop_type(prop_type):
  if prop_type == 'passing_yards':
    return [
      'completions', 'attempts', 'passing_air_yards',
      'passing_yards_after_catch', 'passing_tds',
      'sacks', 'passing_epa', 'week'
    ]
  elif prop_type == 'rushing_yards':
    return [
      'carries', 'rushing_first_downs', 'rushing_epa',
      'rushing_tds', 'week'
    ]
  elif prop_type == 'receiving_yards':
    return [
      'receptions', 'targets', 'receiving_air_yards',
      'receiving_yards_after_catch', 'receiving_tds',
      'target_share', 'air_yards_share', 'week'
    ]
  return []

def advanced_player_stats(player, opp_team, prop_type):
  player_data = weekly_data[weekly_data['player_display_name'] == player]
  p_opp_data = player_data[player_data['opponent_team'] == opp_team]

  features = get_features_for_prop_type(prop_type)
  if not features:
    return "Unsupported prop type"

  features.append(prop_type)
  return p_opp_data[features]

#Linear Regression
def advanced_linear_regression(player_data, prop_type, prop_num):
  if len(player_data) < 1:
    return "Not enough historical data"

  features = get_features_for_prop_type(prop_type)

  x = player_data[features]
  y = player_data[prop_type]

  scaler = StandardScaler()
  x_scaled = scaler.fit_transform(x)

  model = LinearRegression()
  model.fit(x_scaled, y)

  prediction = model.predict(scaler.transform(x.iloc[-1:]))[0]
  confidence = abs(0.5 - (prediction - prop_num) / prop_num) * 100

  coefficients = dict(zip(features, model.coef_))

  return {
    'prediction': 'OVER' if prediction > prop_num else 'UNDER',
    'confidence': f"{min(confidence, 100):.1f}%",
    'predicted_yards': f"{prediction:.1f}",
    'average_yards': f"{np.mean(y):.1f}",
    'last_5_avg': f"{np.mean(y[-5:]):.1f}",
    'number_of_games': len(y),
    'feature_importance': coefficients
  }

#Random Forest
def advanced_random_forest(player_data, prop_type, prop_num, n_trees=100):
  if len(player_data) < 1:
    return "Not enough historical data"

  features = get_features_for_prop_type(prop_type)

  X = player_data[features]
  y = player_data[prop_type]

  rf = RandomForestRegressor(n_estimators=n_trees)
  rf.fit(X, y)

  prediction = rf.predict(X.iloc[-1:].values)[0]
  confidence = abs(0.5 - (prediction - prop_num) / prop_num) * 100

  importance = dict(zip(features, rf.feature_importances_))

  return {
    'prediction': 'OVER' if prediction > prop_num else 'UNDER',
    'confidence': f"{min(confidence, 100):.1f}%",
    'predicted_yards': f"{prediction:.1f}",
    'average_yards': f"{np.mean(y):.1f}",
    'last_5_avg': f"{np.mean(y[-5:]):.1f}",
    'number_of_games': len(y),
    'feature_importance': importance
  }

#XGBoost
def advanced_xgboost(player_data, prop_type, prop_num):
  if len(player_data) < 1:
    return "Not enough historical data"

  features = get_features_for_prop_type(prop_type)

  X = player_data[features]
  y = player_data[prop_type]

  model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5
  )
  model.fit(X, y)

  prediction = model.predict(X.iloc[-1:].values)[0]
  confidence = abs(0.5 - (prediction - prop_num) / prop_num) * 100

  importance = dict(zip(features, model.feature_importances_))

  return {
    'prediction': 'OVER' if prediction > prop_num else 'UNDER',
    'confidence': f"{min(confidence, 100):.1f}%",
    'predicted_yards': f"{prediction:.1f}",
    'average_yards': f"{np.mean(y):.1f}",
    'last_5_avg': f"{np.mean(y[-5:]):.1f}",
    'number_of_games': len(y),
    'feature_importance': importance
  }

#Neural Network
def advanced_neural_network(player_data, prop_type, prop_num):
  if len(player_data) < 1:
    return "Not enough historical data"

  features = get_features_for_prop_type(prop_type)

  X = player_data[features]
  y = player_data[prop_type]

  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)

  model = Sequential([
    Dense(64, activation='relu', input_shape=(len(features),)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
  ])

  model.compile(optimizer='adam', loss='mse')

  model.fit(X_scaled, y, epochs=100, verbose=0)

  prediction = model.predict(scaler.transform(X.iloc[-1:]))[0][0]
  confidence = abs(0.5 - (prediction - prop_num) / prop_num) * 100

  return {
    'prediction': 'OVER' if prediction > prop_num else 'UNDER',
    'confidence': f"{min(confidence, 100):.1f}%",
    'predicted_yards': f"{prediction:.1f}",
    'average_yards': f"{np.mean(y):.1f}",
    'last_5_avg': f"{np.mean(y[-5:]):.1f}",
    'number_of_games': len(y)
  }

def compare_advanced_models():
  print("Example: Brock Purdy, SEA, passing_yards, 262.5")

  player = input('Player Name: ')
  opp_team = input('Opponent Team: ')
  prop_type = input('Prop Type: ')
  prop_num = float(input('Prop Number: '))

  data = advanced_player_stats(player, opp_team, prop_type)
  if isinstance(data, str):
    return data

  models = {
    'Multiple Linear Regression': advanced_linear_regression,
    'Random Forest': advanced_random_forest,
    'XGBoost': advanced_xgboost,
    'Neural Network': advanced_neural_network
  }

  print("\nPredictions from different models:")
  for name, model_func in models.items():
    result = model_func(data, prop_type, prop_num)
    print(f"\n{name}:")
    for key, value in result.items():
      print(f"{key}: {value}")

#Main Method

In [ ]:
compare_models()

Example: Brock Purdy, SEA, passing_yards, 262.5
Player Name: Joe Burrow
Opponent Team: DEN
Prop Type: passing_yards
Prop Number: 279.5

Predictions from different models:

KNN:
prediction: UNDER
confidence: 100.0%
historical_accuracy: 100.0%
average_yards: 157.0
last_5_avg: 157.0
times_over: 0 out of 1 games
number_of_games: 1
k_used: 1

Linear Regression:
prediction: UNDER
confidence: 93.8%
predicted_yards: 157.0
average_yards: 157.0
last_5_avg: 157.0
number_of_games: 1

Random Forest:
prediction: UNDER
confidence: 93.8%
predicted_yards: 157.0
average_yards: 157.0
last_5_avg: 157.0
number_of_games: 1

SVM:
prediction: UNDER
confidence: 93.8%
predicted_yards: 157.0
average_yards: 157.0
last_5_avg: 157.0
number_of_games: 1


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step

Neural Network:
prediction: UNDER
confidence: 100.0%
predicted_yards: 104.3
average_yards: 157.0
last_5_avg: 157.0
number_of_games: 1


In [ ]:
compare_advanced_models()

Example: Brock Purdy, SEA, passing_yards, 262.5
Player Name: Brock Purdy
Opponent Team: SEA
Prop Type: passing_yards
Prop Number: 262.5

Predictions from different models:

Multiple Linear Regression:
prediction: UNDER
confidence: 89.4%
predicted_yards: 159.0
average_yards: 256.7
last_5_avg: 264.6
number_of_games: 6
feature_importance: {'completions': 0.8335618665349216, 'attempts': -8.029450902285344, 'passing_air_yards': 29.086133917012585, 'passing_yards_after_catch': 48.706719161814576, 'passing_tds': 4.233101412442942, 'sacks': 25.554947208768215, 'passing_epa': 7.461037082349567, 'week': -23.046052085353477}

Random Forest:
prediction: UNDER
confidence: 81.1%
predicted_yards: 180.8
average_yards: 256.7
last_5_avg: 264.6
number_of_games: 6
feature_importance: {'completions': 0.011925655990105857, 'attempts': 0.028695662809483626, 'passing_air_yards': 0.20912993279758144, 'passing_yards_after_catch': 0.21820609822195614, 'passing_tds': 0.07573246672674422, 'sacks': 0.13246423898182

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step

Neural Network:
prediction: UNDER
confidence: 100.0%
predicted_yards: 33.2
average_yards: 256.7
last_5_avg: 264.6
number_of_games: 6
